In [1]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn-darkgrid')
warnings.filterwarnings("ignore")

In [19]:
path = 'C:/Users/theod/Google Drive/Stevens/Fall 2020/FE 690 - Machine Learning in Finance/Projects/'
Df = pd.read_csv(path + 'Project_1_v1/stock_price_minutes.csv', index_col=0)
Df.head(5)

date_index      date    time  AAPL_open  AAPL_high  AAPL_low  AAPL_close  \
0           0  20180102  173100     170.18     170.19    169.88      169.94   
1           1  20180102  173200     169.90     169.93    169.67      169.69   
2           2  20180102  173300     169.67     169.88    169.58      169.58   
3           3  20180102  173400     169.46     169.46    169.27      169.33   
4           4  20180102  173500     169.45     169.60    169.44      169.59   

   AAPL_vol  BAC_open  BAC_high  ...  T_open  T_high  T_low  T_close  T_vol  \
0     17654     29.80    29.800  ...   39.03   39.03  39.03    39.03      0   
1      8200     29.71    29.760  ...   39.03   39.03  38.97    39.00   1000   
2      9380     29.77    29.795  ...   39.00   39.03  39.00    39.02   5376   
3      5348     29.77    29.770  ...   39.02   39.02  38.97    38.97   5137   
4      9700     29.74    29.745  ...   38.97   39.01  38.97    38.98   5499   

   XOM_open  XOM_high  XOM_low  XOM_close  XOM_vol  
0     83.82     83.84    83.77      83.77      800  
1     83.80     83.94    83.80      83.94     1391  
2     83.90     83.93    83.84      83.90     9400  
3     83.87     83.90    83.82      83.86     8728  
4     83.88     83.89    83.79      83.79     5430  

[5 rows x 78 columns]

In [21]:
# Predictors
Df['AAPL_Open-Close'] = Df.AAPL_open - Df.AAPL_close
Df['AAPL_High-Low'] = Df.AAPL_high - Df.AAPL_low
X = Df[['AAPL_Open-Close', 'AAPL_High-Low']]
X.head()

AAPL_Open-Close  AAPL_High-Low
0             0.24           0.31
1             0.21           0.26
2             0.09           0.30
3             0.13           0.19
4            -0.14           0.16

In [23]:
# Targets
y = np.where(Df['AAPL_close'].shift(-1) > Df['AAPL_close'], 1, 0)

In [24]:
split_percentage = 0.8
split = int(split_percentage*len(Df))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

In [25]:
cls = SVC().fit(X_train, y_train, verbose = True)

In [26]:
accuracy_train = accuracy_score(y_train, cls.predict(X_train))
accuracy_test = accuracy_score(y_test, cls.predict(X_test))
print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))
print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))


Train Accuracy: 52.65%
Test Accuracy: 51.86%


In [27]:
# Predicted Signal
Df['Predicted_Signal'] = cls.predict(X)
# Daily returns
Df['Return'] = Df.Close.pct_change()
# Strategy returns
Df['Strategy_Return'] = Df.Return * Df.Predicted_Signal.shift(1)
# Geometric returns
geometric_returns = (Df.Strategy_Return.iloc[split:]+1).cumprod()
# Plot Geometric returns
geometric_returns.plot(figsize=(10, 7),color='g')
plt.ylabel("Strategy Returns (%)")
plt.xlabel("Date")
plt.show()

AttributeError: 'DataFrame' object has no attribute 'Close'